<a href="https://colab.research.google.com/github/Kawin101/935442-ImageProcessing/blob/main/Lab10_(7_3_2023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.2 Image recognition_AI

In [ ]:
# Do you know VGG16?
# https://www.mygreatlearning.com/blog/introduction-to-vgg16/
# https://www.geeksforgeeks.org/vgg-16-cnn-model/
# https://viso.ai/deep-learning/vgg-very-deep-convolutional-networks/
# https://datagen.tech/guides/computer-vision/vgg16/

In [ ]:
# Imagenet 1000 Class List
# https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import argparse
import cv2

inputShape=(224,224)
preprocess = imagenet_utils.preprocess_input
image = load_img('/content/drive/MyDrive/3-2Y2023_Second/935442_Image-Processing/VGG16/BlueBerry/01.jpg', target_size=inputShape)
img = load_img('/content/drive/MyDrive/3-2Y2023_Second/935442_Image-Processing/VGG16/BlueBerry/01.jpg', target_size=inputShape)
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = preprocess(image)
model = VGG16(weights="imagenet")
predict = model.predict(image)
P = imagenet_utils.decode_predictions(predict)
display(img)
for (i,(imagenetID,label,prob)) in enumerate(P[0]):
  print("{}.{}:{:.2f}%".format(i+1,label,prob*100))

# 5.3 Image recognition_Custom Object

In [ ]:
# 20 Image with different 2 Type
# https://teachablemachine.withgoogle.com/

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = VGG16(include_top=False, input_shape=(224,224,3))
for layer in model.layers:
  layer.trainable = False

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model = Model(inputs=model.inputs, outputs=output)
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(featurewise_center=True)
datagen.mean=[123.68, 116.779, 103.939]
train_it = datagen.flow_from_directory('/content/drive/MyDrive/3-2Y2023_Second/935442_Image-Processing/training/', 
                                       class_mode='binary',batch_size=16,target_size=(224,224))

model.fit(train_it,steps_per_epoch=len(train_it),epochs=100,verbose=0)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

testImg = load_img('/content/drive/MyDrive/3-2Y2023_Second/935442_Image-Processing/test/avotest01.jpg',target_size=(224,224))
img = img_to_array(testImg)
img = img.reshape(1,224,224,3)
img = img.astype('float32')
img = img- [123.68, 116.779, 103.939]
result = model.predict(img)
print(result)